In [23]:
using NCDatasets
using DIVAnd
using JupyterFormatter
enable_autoformat()
using Dates
using JSON3
using HTTP
using OrderedCollections
using CairoMakie, Makie
include("./DIVAndFairEase.jl")

Main.DIVAndFairEase

In [2]:
token = ENV["beaconAPItoken"];

In [19]:
regionname = "North_Adriatic"
varname = "sea_water_temperature"
# varname = "sea_water_salinity"
doplot = true
outputdir = "../data/testQC"
mkpath(outputdir)

datasourcelist = [
    "Euro-Argo",
    "CORA Profile",
    "CORA Timeseries",
    "World Ocean Database",
    "SeaDataNet CDI TS",
];

In [5]:
domaininfo = Dict(
    "North_Adriatic" => [12.0, 18.0, 43.0, 46.0],
    "Arctic_region" => [-44.25, 70.0, 56.5, 83.0],
    "North_East_Atlantic" => [-42.0, -0.1, 24.9, 48.0],
    "Baltic_Sea" => [9.4, 30.9, 53.0, 65.9],
    "Black_Sea" => [26.5, 41.95, 40.0, 47.95],
    "Mediterranean_Sea" => [-7.0, 36.375, 30.0, 45.875],
    "North_Sea" => [-100.0, 50.0, -80.0, 80.0],
    "Canary_Islands" => [-20.0, -9.0, 25.0, 31.5],
    "World_Ocean" => [-180.0, 180.0, -90.0, 90.0],
);

In [6]:
mindepth = 0.0 #Minimum water depth
maxdepth = 1000.0 #Maximum water depth
datestart = DateTime(1960, 1, 1)
dateend = DateTime(2024, 12, 31)

2024-12-31T00:00:00

In [7]:
minlon = domaininfo[regionname][1]
maxlon = domaininfo[regionname][2]
minlat = domaininfo[regionname][3]
maxlat = domaininfo[regionname][4]

46.0

## Argo

In [21]:
datasource = "Euro-Argo"
queryparams = [
    OrderedDict(
        "column_name" => "TEMP_ADJUSTED",
        "alias" => "Temperature",
        "skip_fill_values" => true,
    ),
    OrderedDict("column_name" => "TEMP_ADJUSTED_QC"),
    OrderedDict("column_name" => "TEMP_ADJUSTED_QC.long_name"),
    OrderedDict("column_name" => "TEMP_ADJUSTED_QC.conventions"),
    OrderedDict("column_name" => "JULD", "alias" => "datetime"),
    OrderedDict("column_name" => "PRES", "alias" => "DEPTH"),
    OrderedDict("column_name" => "LONGITUDE", "alias" => "LONGITUDE"),
    OrderedDict("column_name" => "LATITUDE", "alias" => "LATITUDE"),
]

filters = [
    OrderedDict(
        "for_query_parameter" => "datetime",
        "min" => Dates.format(datestart, "yyyy-mm-ddT00:00:00"),
        "max" => Dates.format(dateend, "yyyy-mm-ddT00:00:00"),
    ),
    OrderedDict("for_query_parameter" => "DEPTH", "min" => mindepth, "max" => maxdepth),
    OrderedDict("for_query_parameter" => "LONGITUDE", "min" => minlon, "max" => maxlon),
    OrderedDict("for_query_parameter" => "LATITUDE", "min" => minlat, "max" => maxlat),
    OrderedDict("for_query_parameter" => "Temperature", "min" => 0, "max" => 40.0),
]

paramdict = OrderedDict(
    "query_parameters" => queryparams,
    "filters" => filters,
    "output" => Dict("format" => "netcdf"),
)
query = JSON3.write(paramdict);

In [22]:
filename = joinpath(outputdir, "$(datasource).nc")

@time open(filename, "w") do io
    r = HTTP.request(
        "POST",
        joinpath(DIVAndFairEase.beacon_services[datasource], "api/query"),
        [
            "Content-type" => "application/json",
            "Authorization" => "Bearer $(token)",
        ],
        query,
        response_stream = io,
    )
    @info(r.status)
end

LoadError: UndefVarError: `HTTP` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: HTTP is loaded but not imported in the active module Main.